In [3]:
def read_sent(path):
    ents = []
    curEnts = []
    for line in open(path):
        line = line.strip()
        if line == '':
            ents.append(curEnts)
            curEnts = []
        elif line[0] == '#' and len(line.split('\t')) == 1:
            continue
        else:
            curEnts.append(line.split('\t')[1])
    return(ents)

def read_labels(path):
    ents = []
    curEnts = []
    for line in open(path):
        line = line.strip()
        if line == '':
            ents.append(curEnts)
            curEnts = []
        elif line[0] == '#' and len(line.split('\t')) == 1:
            continue
        else:
            curEnts.append(line.split('\t')[2])
    return(ents)

def read_index(path):
    ents = []
    curEnts = []
    for line in open(path):
        line = line.strip()
        if line == '':
            ents.append(curEnts)
            curEnts = []
        elif line[0] == '#' and len(line.split('\t')) == 1:
            continue
        else:
            curEnts.append(line.split('\t')[0])
    return(ents)

In [4]:
#Training data

#returns list of lists
training_labels = read_labels("baseline_model/baseline_data/en_ewt-ud-train.iob2")
training_sent = read_sent("baseline_model/baseline_data/en_ewt-ud-train.iob2")

#flatten to one list to be able to use myutils
train_labels = sum(training_labels, [])
train_sent = sum(training_sent, [])

In [5]:
#Evaluation data

dev_labels = read_labels("baseline_model/baseline_data/en_ewt-ud-dev.iob2")
dev_sent = read_sent("baseline_model/baseline_data/en_ewt-ud-dev.iob2")

dev_flat_labels = sum(dev_labels, [])
dev_flat_sent = sum(dev_sent, [])

In [6]:
#Test data
#Keeping track of indeces to save to required .iob2 format for model's predictions

test_labels = read_labels("baseline_model/baseline_data/en_ewt-ud-test.iob2")
test_sent = read_sent("baseline_model/baseline_data/en_ewt-ud-test.iob2")
test_index = read_index("baseline_model/baseline_data/en_ewt-ud-test.iob2")

test_flat_labels = sum(test_labels, [])
test_flat_sent = sum(test_sent, [])
test_flat_index = sum(test_index, [])

In [7]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [8]:
inputs = []
for sentence in training_sent:
    inputt = tokenizer(sentence, is_split_into_words=True)
    inputs.append(inputt)

In [9]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [38]:
import myutils
UNK = "[UNK]"

id2label, label2id = myutils.labels2lookup(train_labels, UNK)
NLABELS = len(id2label)
print(train_labels)
print(label2id)

#converting BIO labels to numerical labels
train_labels_num = [label2id.get(label, label2id[UNK]) for label in train_labels]

['O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 

In [39]:
labels = training_labels
word_ids = []
for i in inputs:
    word_ids.append(i.word_ids())
word_ids_flat = sum(word_ids, [])
aligned_training = align_labels_with_tokens(train_labels_num, word_ids_flat)


In [40]:
def list_to_sentences(lst):
    sentences = []
    current_sentence = []
    
    for item in lst:
        if item == -100:
            if current_sentence:
                sentences.append(current_sentence)
                current_sentence = []
        else:
            current_sentence.append(item)
    
    if current_sentence:
        sentences.append(current_sentence)
    
    return sentences


sentences = list_to_sentences(aligned_training)


In [41]:
to_zip_train = [[-100] + sublist + [-100] for sublist in sentences]

In [42]:
# Assuming inputs and to_zip_train are defined as provided

# Zip inputs with to_zip_train and add 'label' key to each item in inputs
for input_item, label_item in zip(inputs, to_zip_train):
    input_item['labels'] = label_item

# Print the updated inputs list
print(inputs[0])


{'input_ids': [101, 2777, 1107, 1103, 1362, 1110, 146, 13855, 10337, 136, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 1, 1, 1, 1, 1, 2, 2, 2, 1, -100]}


In [43]:
inputs_dev = []
for sentence in dev_sent:
    inputt = tokenizer(sentence, is_split_into_words=True)
    inputs_dev.append(inputt)

In [44]:
dev_ner_labels = [label2id.get(label, label2id[UNK]) for label in dev_flat_labels]

In [45]:
word_ids = []
for i in inputs_dev:
    word_ids.append(i.word_ids())
word_ids_flat = sum(word_ids, [])
aligned_dev = align_labels_with_tokens(dev_ner_labels, word_ids_flat)

In [46]:
to_zip_dev = [[-100] + sublist + [-100] for sublist in list_to_sentences(aligned_dev)]

In [47]:
for input_item, label_item in zip(inputs_dev, to_zip_dev):
    input_item['labels'] = label_item

In [48]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [49]:
import evaluate

metric = evaluate.load("seqeval")

In [50]:
id2label = {v: k for k, v in label2id.items()}

In [52]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label = id2label,
    label2id = label2id,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['cl

In [54]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [56]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=inputs,
    eval_dataset=inputs_dev,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

  0%|          | 0/4704 [00:00<?, ?it/s]

{'loss': 0.0778, 'learning_rate': 1.7874149659863948e-05, 'epoch': 0.32}
{'loss': 0.0667, 'learning_rate': 1.5748299319727894e-05, 'epoch': 0.64}
{'loss': 0.057, 'learning_rate': 1.3622448979591838e-05, 'epoch': 0.96}


  0%|          | 0/251 [00:00<?, ?it/s]

{'eval_loss': 1.8575029373168945, 'eval_runtime': 63.8962, 'eval_samples_per_second': 31.316, 'eval_steps_per_second': 3.928, 'epoch': 1.0}
{'loss': 0.0507, 'learning_rate': 1.1496598639455783e-05, 'epoch': 1.28}
{'loss': 0.0477, 'learning_rate': 9.370748299319729e-06, 'epoch': 1.59}
{'loss': 0.0399, 'learning_rate': 7.244897959183675e-06, 'epoch': 1.91}


  0%|          | 0/251 [00:00<?, ?it/s]

{'eval_loss': 2.010057210922241, 'eval_runtime': 63.0284, 'eval_samples_per_second': 31.748, 'eval_steps_per_second': 3.982, 'epoch': 2.0}
{'loss': 0.0319, 'learning_rate': 5.119047619047619e-06, 'epoch': 2.23}
{'loss': 0.0263, 'learning_rate': 2.993197278911565e-06, 'epoch': 2.55}
{'loss': 0.0248, 'learning_rate': 8.673469387755103e-07, 'epoch': 2.87}


  0%|          | 0/251 [00:00<?, ?it/s]

{'eval_loss': 2.164790630340576, 'eval_runtime': 87.8427, 'eval_samples_per_second': 22.779, 'eval_steps_per_second': 2.857, 'epoch': 3.0}
{'train_runtime': 5989.9964, 'train_samples_per_second': 6.282, 'train_steps_per_second': 0.785, 'train_loss': 0.0459354202560827, 'epoch': 3.0}


TrainOutput(global_step=4704, training_loss=0.0459354202560827, metrics={'train_runtime': 5989.9964, 'train_samples_per_second': 6.282, 'train_steps_per_second': 0.785, 'train_loss': 0.0459354202560827, 'epoch': 3.0})

In [ ]:
import pickle

# Get the trained model
trained_model = trainer.model

# Save the trained model as a pickle file
with open("trained_model.pkl", "wb") as f:
    pickle.dump(trained_model, f)


In [ ]:
inputs_test = []
for sentence in test_sent:
    inputt = tokenizer(sentence, is_split_into_words=True)
    inputs_test.append(inputt)

In [ ]:
test_labels_num = [label2id.get(label, label2id[UNK]) for label in test_flat_labels]

In [ ]:
word_ids = []
for i in inputs_test:
    word_ids.append(i.word_ids())
word_ids_flat = sum(word_ids, [])
aligned_test = align_labels_with_tokens(test_labels_num, word_ids_flat)

In [ ]:
to_zip_test = [[-100] + sublist + [-100] for sublist in list_to_sentences(aligned_test)]

In [ ]:
for input_item, label_item in zip(inputs_test, to_zip_test):
    input_item['labels'] = label_item

In [ ]:
import pickle
from transformers import Trainer, TrainingArguments
import numpy as np

# Load the trained model from the .pkl file
with open("trained_model.pkl", "rb") as f:
    trained_model = pickle.load(f)

# Define a function to predict labels using the model
def predict_labels(model, test_dataset):
    # Define a trainer with a dummy training arguments object
    trainer = Trainer(
        model=model,
        args=TrainingArguments("dummy"),
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

    # Make predictions on the test dataset
    predictions = trainer.predict(test_dataset)
    
    # Convert logits to predicted labels
    predicted_labels = np.argmax(predictions.predictions, axis=-1)
    
    return predicted_labels

# Use the predict_labels function to get predictions on the test data
test_predictions = predict_labels(trained_model, inputs_test)

# Save the predictions to a .pkl file
with open("test_predictions.pkl", "wb") as f:
    pickle.dump(test_predictions, f)
